In [ ]:
# Code for BRATS 2019 dataset

from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout, Input, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow as tf
from keras import optimizers
import matplotlib.pyplot as plt
import skimage.io as io
from glob import glob
import numpy as np
import random as r
import cv2
from sklearn.model_selection import train_test_split
import gc
import time
from contextlib import redirect_stdout
from glob import glob
import os
import imageio
from keras.losses import binary_crossentropy

#Start Execution Time
start_time = time.time()

path_in = "..."

out_folder = 'Run_UNet++_3D/'
path_out = "Outputs/" + out_folder


if not os.path.exists(path_out):
    os.makedirs(path_out)
    print("Directory Created")
else:    
    print("Directory Already Exists") 
    
files = glob(path_in + "**/*.nii.gz", recursive = True)

print("No. of files : ", len(files))
    

def img_to_array(path, end):
    
    # get locations
    files = glob(path+end, recursive=True)
    
    img_list = []
    
    #r.seed(42)
    #r.shuffle(files)
    
    for file in files:
        img = io.imread(file, plugin="simpleitk")

        # standardization
        img = (img-img.mean())/img.std()
        img.astype("float32")
        
        for slice in range(60, 130):
            img_s = img[slice,:,:]
            
            # resize
            img_s = cv2.resize(img_s, (128,128))
            
            img_s = np.expand_dims(img_s, axis=0)
            img_list.append(img_s)
            
    return np.array(img_list,np.float32)

def seg_to_array(path, end, label):
    
    # get locations
    files = glob(path+end, recursive=True)
    
    img_list = []
    
    #r.seed(42)
    #r.shuffle(files)
    
    for file in files:
        img = io.imread(file, plugin="simpleitk")
        
        # all tumor
        if label == 1:
            img[img != 0] = 1
        
        # Non-enhancing Tumor
        if label == 2:
            img[img != 1] = 0
        
        # Without Edema
        if label == 3:
            img[img == 2] = 0
            img[img != 0] = 1
        
        # Enhancing Tumor
        if label == 4:
            img[img != 4] = 0
            img[img == 4] = 1
            
        img.astype("float32")
        
        for slice in range(60, 130):
            img_s = img[slice,:,:]
            
            # resize
            img_s = cv2.resize(img_s, (128,128))
            
            img_s = np.expand_dims(img_s, axis=0)
            img_list.append(img_s)
            
    return np.array(img_list,np.float32)

def dice_coef(y_true, y_pred):
    print("y_true DCC : ", y_true)
    print("y_pred DCC : ", y_pred)
    smooth = 0.005 
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    print("y_true loss : ", y_true)
    print("y_pred loss : ", y_pred)
    return 1-dice_coef(y_true, y_pred)

# Function for a Convolution layer containing 2 blocks
def conv2d_block( input_tensor, n_filters, kernel_size = (3,3), name="contraction"):
  "Add 2 conv layer"
  x = Conv2D(filters=n_filters, kernel_size=kernel_size, kernel_initializer='he_normal', 
             padding='same',activation="relu", name=name+'_1')(input_tensor)
  
  x = Conv2D(filters=n_filters, kernel_size=kernel_size, kernel_initializer='he_normal', 
             padding='same',activation="relu",name=name+'_2')(x)
  return x

t1 = img_to_array(path=path, end="**/*t1.nii.gz")
t1ce = img_to_array(path=path, end="**/*t1ce.nii.gz")
t2 = img_to_array(path=path, end="**/*t2.nii.gz")
flair = img_to_array(path=path, end="**/*flair.nii.gz")

seg = seg_to_array(path=path, end="**/*seg.nii.gz", label=1)

print("t1 shape : ", t1.shape) 
print("t1ce shape : ", t1ce.shape) 
print("t2 shape : ", t2.shape) 
print("flair shape : ", flair.shape)

print("seg shape : ", seg.shape)

X = np.concatenate((t1, t1ce, t2, flair), axis=1)
print("X_train shape : ", X.shape)
print("X_train datatype : ", X.dtype)

# Training Test division
X_train,X_test,Y_train,Y_test = train_test_split(X,seg,test_size=0.2)
del X
del seg
gc.collect()
print("X_train shape : ", X_train.shape)
print("X_train datatype : ", X_train.dtype)
print("X_train, X_test Done.")

# Training is divided into training and validation
X_train1,X_val,Y_train1,Y_val = train_test_split(X_train,Y_train,test_size=0.2)

file0 = open(path_out + 'Data_division.txt','w')
file0.write("Training Data: " + str(Y_train.shape)) 
file0.write("\n")
file0.write("Test Data: " + str(Y_test.shape)) 
file0.write("\n")
file0.write("Final Training Data: " + str(Y_train1.shape)) 
file0.write("\n")
file0.write("Validation Data: " + str(Y_val.shape)) 
file0.close()

del X_train
del Y_train
gc.collect()

print("X_train1, X_test1 Done.")
    

K.set_image_data_format('channels_first')

def unetpp():
    
    inputs = Input((4, 128 , 128))
    # ------------------------------------------------------------------------
    # X(0,0)
    d1 = conv2d_block(inputs, 64, name="contraction_1") # None, 64, 128, 128
    #print("d1: ", d1) 
    p1 = MaxPooling2D( pool_size=(2,2), strides=(2,2), padding='same')(d1)
    p1 = BatchNormalization(momentum=0.8)(p1)
    p1 = Dropout(0.1)(p1) # (None, 64, 64, 64)
    #print("p1:", p1)

    # X(1,0)
    d2 = conv2d_block( p1, 128, name="contraction_2_1" )
    #print("d2: ", d2)
    p2 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(d2)
    p2 = BatchNormalization(momentum=0.8)(p2)
    p2 = Dropout(0.1)(p2) # (None, 128, 32, 32)
    #print("p2:", p2)

    # X(2,0)
    d3 = conv2d_block( p2, 256, name="contraction_3_1")
    #print("d3: ", d3)
    p3 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(d3)
    p3 = BatchNormalization(momentum=0.8)(p3)
    p3 = Dropout(0.1)(p3) # (None, 256, 16, 16)
    #print("p3:", p3)

    # X(3,0)
    d4 = conv2d_block(p3,512, name="contraction_4_1")
    #print("d4: ", d4)
    p4 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(d4)
    p4 = BatchNormalization(momentum=0.8)(p4)
    p4 = Dropout(0.1)(p4)
    #print("p4:", p4) # (None, 512, 8, 8)

    # X(4,0)
    d5 = conv2d_block(p4,512, name="contraction_5_1")
    #print("d5:", d5) # (None, 512, 8, 8)


    # ------------------------------------------------------------------------
    # ---------------- UNet++ L1 ---------------
    # X(0,1)
    du_01 = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'same')(d2)
    #print("du_01: ", du_01) # 64, 128, 128
    du_01 = concatenate([du_01,d1], axis=1) # d1 = None, 64, 128, 128
    #print("du_01 concat: ", du_01) # None, 64, 128, 256
    du_01 = Dropout(0.1)(du_01)
    c_01 = conv2d_block(du_01, 64, name="expansion_01")
    #print("c_01: ", c_01)

    # ---------------- UNet++ L2 ---------------
    # X(1,1)
    du_11 = Conv2DTranspose(128, (3, 3), strides = (2, 2), padding = 'same')(d3)
    du_11 = concatenate([du_11,d2], axis=1)
    du_11 = Dropout(0.1)(du_11)
    c_11 = conv2d_block(du_11, 128, name="expansion_11")
    #print("c_11: ", c_11)

    # X(0,2)
    du_02 = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'same')(c_11)
    du_02 = concatenate([du_02,du_01,d1], axis=1)
    du_02 = Dropout(0.1)(du_02)
    c_02 = conv2d_block(du_02, 64, name="expansion_02")
    #print("c_02: ", c_02)

    # ---------------- UNet++ L3 ---------------
    # X(2,1)
    du_21 = Conv2DTranspose(256, (3, 3), strides = (2, 2), padding = 'same')(d4)
    du_21 = concatenate([du_21,d3], axis=1)
    du_21 = Dropout(0.1)(du_21)
    c_21 = conv2d_block(du_21, 256, name="expansion_21")
    #print("c_21: ", c_21)

    # X(1,2)
    du_12 = Conv2DTranspose(128, (3, 3), strides = (2, 2), padding = 'same')(c_21)
    du_12 = concatenate([du_12,c_11,d2], axis=1)
    du_12 = Dropout(0.1)(du_12)
    c_12 = conv2d_block(du_12, 128, name="expansion_12")
    #print("c_12: ", c_12)

    # X(0,3)
    du_03 = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'same')(c_12)
    du_03 = concatenate([du_03,c_02,c_01,d1], axis=1)
    du_03 = Dropout(0.1)(du_03)
    c_03 = conv2d_block(du_03, 64, name="expansion_03")
    #print("c_03: ", c_03)

    # ---------------- UNet++ L4 ---------------
    # X(3,1)
    u_31 = Conv2DTranspose(512, (3, 3), strides = (2, 2), padding = 'same')(d5)
    u_31 = concatenate([u_31,d4], axis=1)
    u_31 = Dropout(0.1)(u_31)
    c_31 = conv2d_block(u_31, 256, name="expansion_1")
    #print("c_31: ", c_31)

    # X(2,2)
    u_22 = Conv2DTranspose(256, (3, 3), strides = (2, 2), padding = 'same')(c_31)
    u_22 = concatenate([u_22,c_21,d3], axis=1)
    u_22 = Dropout(0.1)(u_22)
    c_22 = conv2d_block(u_22, 256, name="expansion_2")
    #print("c_22: ", c_22)

    # X(1,3)
    u_13 = Conv2DTranspose(128, (3, 3), strides = (2, 2), padding = 'same')(c_22)
    u_13 = concatenate([u_13,c_12,c_11,d2], axis=1)
    u_13 = Dropout(0.1)(u_13)
    c_13 = conv2d_block(u_13, 128, name="expansion_3")
    #print("c_13: ", c_13)

    # X(0,4)
    u_04 = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'same')(c_13)
    u_04 = concatenate([u_04,c_03,c_02,c_01,d1], axis=1)
    u_04 = Dropout(0.1)(u_04)
    c_04 = conv2d_block(u_04, 64, name="expansion_4")
    #print("c_04: ", c_04)

    # ------------------------------------------------------------------------
    # Final concatenation
    c_L = concatenate([c_01,c_02,c_03,c_04], axis=1)
    #print("c_L: ", c_L)

    out = Conv2D(1, (1,1), name="output", activation='sigmoid')(c_L)
    
    #print("inputs: ", inputs)
    #print("out: ", out)

    model = Model(inputs=[inputs], outputs=[out])
    
    model.compile(optimizer=optimizers.Adam(lr=1e-4), loss=dice_coef_loss, metrics=['acc', dice_coef])

    # ------------------------------------------------------------------------

    return model


model = unetpp()

with open(path_out + 'modelsummary.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()

model.save_weights(path_out + "model_weights.h5")

hist = model.fit(X_train1, Y_train1, batch_size=8, epochs=50, 
                validation_data=(X_val,Y_val), verbose=1)
print("Model fit Done.")

# Calculating and drawing training plots
acc = hist.history['acc']
val_acc = hist.history['val_acc']
loss = hist.history['loss']
val_loss = hist.history['val_loss']
dice = hist.history['dice_coef']
val_dice = hist.history['val_dice_coef']

avg_acc = np.mean(acc) * 100
avg_val_acc = np.mean(val_acc) * 100
avg_loss = np.mean(loss) * 100
avg_val_loss = np.mean(val_loss) * 100
avg_dcc = np.mean(dice) * 100
avg_val_dcc = np.mean(val_dice) * 100
file0 = open(path_out + 'avg_eval.txt','w')#append mode 
file0.write("Average Training Accuracy: " + str(avg_acc)) 
file0.write("\n")
file0.write("Average Validation Accuracy: " + str(avg_val_acc)) 
file0.write("\n")
file0.write("Average Training DCC: " + str(avg_dcc)) 
file0.write("\n")
file0.write("Average Validation DCC: " + str(avg_val_dcc)) 
file0.write("\n")
file0.write("Average Training Loss: " + str(avg_loss)) 
file0.write("\n")
file0.write("Average Validation Loss: " + str(avg_val_loss)) 
file0.close()

np.savetxt(path_out + 'train_acc.txt', acc, delimiter="\n") 
np.savetxt(path_out + 'val_acc.txt', val_acc, delimiter="\n") 
np.savetxt(path_out + 'train_loss.txt', loss, delimiter="\n") 
np.savetxt(path_out + 'val_loss.txt', val_loss, delimiter="\n") 
np.savetxt(path_out + 'dcc.txt', dice, delimiter="\n") 
np.savetxt(path_out + 'val_dcc.txt', val_dice, delimiter="\n") 

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 4))
t = f.suptitle('Unet++ Performance in Segmenting Tumors', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)
epoch_list = hist.epoch

ax1.plot(epoch_list, hist.history['acc'], label='Train Accuracy')
ax1.plot(epoch_list, hist.history['val_acc'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, epoch_list[-1], 5))
ax1.set_ylabel('Accuracy Value');ax1.set_xlabel('Epoch');ax1.set_title('Accuracy')
ax1.legend(loc="best");ax1.grid(color='gray', linestyle='-', linewidth=0.5)

ax2.plot(epoch_list, hist.history['loss'], label='Train Loss')
ax2.plot(epoch_list, hist.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, epoch_list[-1], 5))
ax2.set_ylabel('Loss Value');ax2.set_xlabel('Epoch');ax2.set_title('Loss')
ax2.legend(loc="best");ax2.grid(color='gray', linestyle='-', linewidth=0.5)

ax3.plot(epoch_list, hist.history['dice_coef'], label='Train DCC')
ax3.plot(epoch_list, hist.history['val_dice_coef'], label='Validation DCC')
ax3.set_xticks(np.arange(0, epoch_list[-1], 5))
ax3.set_ylabel('DCC Value');ax3.set_xlabel('Epoch');ax3.set_title('DCC')
ax3.legend(loc="best");ax3.grid(color='gray', linestyle='-', linewidth=0.5)

plt.savefig(path_out + 'Performance_Plot.jpg')

predicted = model.predict(X_test)
np.save(path_out + 'test_tumor.npy', Y_test)
np.save(path_out + 'predicted_tumor.npy', predicted)

test_eval = model.evaluate(X_test, Y_test, verbose=0)
test_loss = test_eval[0] * 100
test_acc = test_eval[1] * 100
test_dcc = test_eval[2] * 100
print('Test accuracy: ', test_acc)
print('Test loss: ', test_loss)
print('Test dcc: ', test_dcc)

file_t = open(path_out + 'test_eval.txt','w')#append mode 
file_t.write("Test Accuracy: " + str(test_acc)) 
file_t.write("\n")
file_t.write("Test Loss: " + str(test_loss)) 
file_t.write("\n")
file_t.write("Test DCC: " + str(test_dcc)) 
file_t.close() 

pred_path = path_out + 'Prediction/'

if not os.path.exists(pred_path):
    os.makedirs(pred_path)
    print("Directory Created")
else:    
    print("Directory Already Exists") 

i = 0
for idx in range(0,X_test.shape[0]):
    i = i + 1
    imageio.imwrite(pred_path + str(i) + '_MRI.jpg', X_test[idx][0])
    imageio.imwrite(pred_path + str(i) + '_Mask.jpg', Y_test[idx][0])
    imageio.imwrite(pred_path + str(i) + '_PredMask.jpg', predicted[idx][0])
    

# Shows predicted outputs by choosing 10 Random images
plt.figure(figsize=(8,30))
i=1;total=10
t = 0
for idx in np.random.randint(0,high=X_test.shape[0],size=total):
    t = t + 1
    plt.subplot(total,3,i);i+=1
    plt.imshow(X_test[idx][0], cmap="inferno")
    plt.title("MRI");plt.axis('off')
    
    plt.subplot(total,3,i);i+=1
    plt.imshow(Y_test[idx][0], cmap="inferno")
    plt.title("Ground Truth");plt.axis('off')
    
    plt.subplot(total,3,i);i+=1
    plt.imshow(predicted[idx][0], cmap="inferno")
    plt.title("Predicted Tumor");plt.axis('off')

plt.savefig(path_out + 'Sample_Output_Plot.jpg')


#End Time
print("--- Time : %s seconds ---" % (time.time() - start_time))
f_time = open(path_out + 'Time_Info.txt', 'w')
print("%s %f" % ("Execution Time: ", time.time() - start_time), file=f_time)
f_time.close()